![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## This notebook is an example of how to implement NVR triggered by object detection. 
A video stream is processed by the person detection model. Once any person is detected, the notification event is generated and video clip is saved.

This example uses `degirum_tools.streams` streaming toolkit.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on AI accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

The script can use either local camera or web camera connected to the machine, or a video file. The camera index or URL or video file path needs to be specified in the code below by assigning `video_source`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences, model_zoo_url, model names for inference, and video source

In [2]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
#
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
#
# model_name: name of the model for detecting
#
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
#
# zones: list of zones for object tracking
# holdoff_sec: holdoff duration to suppress repeated notifications
# notification_config: configuration for Apprise notifications (see https://github.com/caronc/apprise)
# clip_duration: duration in frames of video clips to save
# storage_config: configuration for object storage to save video clips (can be S3 or local)

import degirum as dg, degirum_tools
from degirum_tools import streams as dgstreams

hw_location = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
video_source = "../../images/WalkingPerson.mp4"
zones = [
    [[450, 100], [950, 100], [950, 600], [450, 600]],
]
holdoff_sec = 3.0
notification_config = "json://console"  # just prints to console
clip_duration = 100
storage_config = degirum_tools.ObjectStorageConfig(
    endpoint="./temp",  # Object storage endpoint URL or local path
    access_key="",  # Access key for the storage account
    secret_key="",  # Secret key for the storage account
    bucket="nvr_clips",  # Bucket name for S3 or local directory name
)

#### The rest of the cells below should run without any modifications

In [3]:
# load model
model = dg.load_model(
    model_name,
    hw_location,
    model_zoo_url,
    degirum_tools.get_token(),
    overlay_show_probabilities=True,
    overlay_line_width=1,
)

#
# create analyzers:
#

anchor = degirum_tools.AnchorPoint.CENTER
window_name = "Live Display (press 'q' to quit)"
event_name = "object_detected"

# object tracker
object_tracker = degirum_tools.ObjectTracker(
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=10,
    anchor_point=anchor,
)

# zone counter
zone_counter = degirum_tools.ZoneCounter(
    zones,
    use_tracking=True,
    triggering_position=[anchor],
    annotation_color=(0, 255, 0),
    window_name=window_name,  # attach display window for interactive zone adjustment
)

# event detector: object in zone
zone_detector = degirum_tools.EventDetector(
    f"""
    Trigger: {event_name}
    when: ZoneCount
    is greater than: 0
    during: [10, frames]
    for at least: [90, percent]
    """,
    show_overlay=False,
)

# event notifier
notifier = degirum_tools.EventNotifier(
    event_name,
    event_name,
    message="{time}: person is detected in zone",
    holdoff=holdoff_sec,
    notification_config=notification_config,
    clip_save=True,
    clip_duration=clip_duration,
    clip_pre_trigger_delay=clip_duration // 2,
    storage_config=storage_config,
)

degirum_tools.attach_analyzers(
    model, [object_tracker, zone_counter, zone_detector, notifier]
)

#
# create gizmos
#

# video source gizmo
cam_source = dgstreams.VideoSourceGizmo(video_source)

# detection gizmo
detector = dgstreams.AiSimpleGizmo(model)

# local display gizmo (just for debugging)
display = dgstreams.VideoDisplayGizmo(window_name, show_ai_overlay=True)

# start composition
dgstreams.Composition(cam_source >> detector >> display).start()